In [3]:
import pandas as pd
import numpy as np

# Загрузка данных
data = pd.read_csv('data_for_spb_hakaton_entities1-Table 1.csv', sep=';')
history = pd.read_csv('history-Table 1.csv', sep=';')
sprints = pd.read_csv('sprints-Table 1.csv', sep=';')


In [11]:
import pandas as pd

# Загрузка данных
data = pd.read_csv('data_for_spb_hakaton_entities1-Table 1.csv', sep=';')
history = pd.read_csv('history-Table 1.csv', sep=';')
sprints = pd.read_csv('sprints-Table 1.csv', sep=';')

# Разделение entity_ids в таблице sprints
sprints['entity_ids'] = sprints['entity_ids'].apply(lambda x: x.strip('{}').split(',') if pd.notnull(x) else [])
sprints_expanded = sprints.explode('entity_ids')
sprints_expanded['entity_id'] = sprints_expanded['entity_ids'].astype(float)

# Соединение data и sprints
data_sprints = pd.merge(data, sprints_expanded, on='entity_id', how='left')

# Соединение с history
final_table = pd.merge(data_sprints, history, on='entity_id', how='left')

# Просмотр результата
final_table.head()


,entity_id,area,type,status,state,priority,ticket_number,name,create_date,created_by,...,sprint_status,sprint_start_date,sprint_end_date,entity_ids,history_property_name,history_date,history_version,history_change_type,history_change,Столбец1
0,94297,Система.Таск-трекер,Дефект,Закрыто,Normal,Средний,PPTS-1965,[FE] Бэклог. Кастомизация колонок. Кастомизаци...,2023-03-16 16:59:00.000000,А. К.,...,Закрыт,2024-08-28 19:00:00.000000,2024-09-10 19:00:00.000000,94297,Время решения 3ЛП ФАКТ,09.10.2024 11:17,1.0,FIELD_CHANGED,<empty> -> 2024-09-10 11:17:06.680223,NaN
1,94297,Система.Таск-трекер,Дефект,Закрыто,Normal,Средний,PPTS-1965,[FE] Бэклог. Кастомизация колонок. Кастомизаци...,2023-03-16 16:59:00.000000,А. К.,...,Закрыт,2024-08-28 19:00:00.000000,2024-09-10 19:00:00.000000,94297,Время решения (ФАКТ),09.10.2024 11:17,1.0,FIELD_CHANGED,<empty> -> 2024-09-10 11:17:06.680223,NaN
2,94297,Система.Таск-трекер,Дефект,Закрыто,Normal,Средний,PPTS-1965,[FE] Бэклог. Кастомизация колонок. Кастомизаци...,2023-03-16 16:59:00.000000,А. К.,...,Закрыт,2024-08-28 19:00:00.000000,2024-09-10 19:00:00.000000,94297,Исполнитель,7/13/23 11:07,1.0,FIELD_CHANGED,user409017mail@mail.com -> user408045mail@mail...,NaN
3,94297,Система.Таск-трекер,Дефект,Закрыто,Normal,Средний,PPTS-1965,[FE] Бэклог. Кастомизация колонок. Кастомизаци...,2023-03-16 16:59:00.000000,А. К.,...,Закрыт,2024-08-28 19:00:00.000000,2024-09-10 19:00:00.000000,94297,Исполнитель,7/21/23 11:06,3.0,FIELD_CHANGED,user4080458@mail.com -> user4091341@mail.com,NaN
4,94297,Система.Таск-трекер,Дефект,Закрыто,Normal,Средний,PPTS-1965,[FE] Бэклог. Кастомизация колонок. Кастомизаци...,2023-03-16 16:59:00.000000,А. К.,...,Закрыт,2024-08-28 19:00:00.000000,2024-09-10 19:00:00.000000,94297,Срок исполнения,7/31/23 15:05,4.0,FIELD_CHANGED,NaN,NaN


In [13]:
data = final_table.copy()

In [18]:
data.columns

Index(['entity_id', 'area', 'type', 'status', 'priority', 'ticket_number',
       'create_date', 'created_by', 'update_date', 'estimation', 'resolution',
       'sprint_name', 'sprint_status', 'sprint_start_date', 'sprint_end_date',
       'entity_ids', 'history_property_name', 'history_date'],
      dtype='object')

In [15]:
columns_to_drop = ['state', 'name', 'updated_by', 'parent_ticket_id', 'assignee', 'owner', 'due_date', 'rank', 'spent', 'workgroup', 'history_version', 'history_change_type', 'history_change', 'Столбец1']

In [16]:
data = data.drop(columns=columns_to_drop)

In [17]:
data.columns

Index(['entity_id', 'area', 'type', 'status', 'priority', 'ticket_number',
       'create_date', 'created_by', 'update_date', 'estimation', 'resolution',
       'sprint_name', 'sprint_status', 'sprint_start_date', 'sprint_end_date',
       'entity_ids', 'history_property_name', 'history_date'],
      dtype='object')

In [22]:
data['priority'].unique()

array(['Средний', 'Критический', 'Высокий', 'Низкий'], dtype=object)

In [30]:
import pandas as pd
import numpy as np

def preprocess(data: pd.DataFrame) -> pd.DataFrame:
    
    """
    Preprocessing + aggregating data
    
    **sprint_name** - Название спринта(по нему происходит агрегация)

    **mean_estimation, median_estimation, sum_estimation** - ['mean', 'median', 'sum']  
    # Средняя, медианная и суммарная оценка времени выполнения задачи (в часах)

    **avg_completion_time** - 'mean',  # Среднее время выполнения задач

    **completion_rate** - # Доля завершённых задач

    **rejected_rate** - # Доля отклонённых задач

    **defects** - # Количество дефектов

    **critical_tasks** - # Количество критических задач

    **task_count** - # Общее количество задач

    return: aggregated data

    """

    data['create_date'] = pd.to_datetime(data['create_date'], errors='coerce')
    data['update_date'] = pd.to_datetime(data['update_date'], errors='coerce')
    data['sprint_start_date'] = pd.to_datetime(data['sprint_start_date'], errors='coerce')
    data['sprint_end_date'] = pd.to_datetime(data['sprint_end_date'], errors='coerce')

    data['completion_time'] = (data['update_date'] - data['create_date']).dt.total_seconds() / 3600  # В часах

    # Метрики для каждого спринта
    agg_sprint_metricks = data.groupby('sprint_name').agg({
        'estimation': ['mean', 'median', 'sum'],  # Средняя, медианная и суммарная оценка
        'status': lambda x: (x.isin(['Закрыто', 'Готово']).sum()) / len(x),  # Доля завершённых задач
        'resolution': lambda x: (x == 'Отклонено').sum() / len(x),  # Доля отклонённых задач
        'completion_time': 'mean',  # Среднее время выполнения задач
        'type': lambda x: (x == 'Дефект').sum(),  # Количество дефектов
        'priority': lambda x: (x == 'Критический').sum(),  # Количество критических задач
        'entity_id': 'count'  # Общее количество задач
    }).reset_index()

    agg_sprint_metricks.columns = ['sprint_name', 'mean_estimation', 'median_estimation', 'sum_estimation',
                            'completion_rate', 'rejected_rate', 'avg_completion_time',
                            'defects', 'critical_tasks', 'task_count']
    
    return agg_sprint_metricks




In [33]:
def _remove_highly_correlated_features(self) -> None:
        corr_matrix = self.df.corr()
        col_corr = set()
        for i in range(len(corr_matrix.columns)):
            for j in range(i):
                if corr_matrix.iloc[i, j] >= self.corr_threshold:
                    colname = corr_matrix.columns[i]
                    col_corr.add(colname)
        self.df.drop(columns=list(col_corr), inplace=True)


In [31]:
agg_sprint_metricks = preprocess(data=data)

In [32]:
agg_sprint_metricks.head()

,sprint_name,mean_estimation,median_estimation,sum_estimation,completion_rate,rejected_rate,avg_completion_time,defects,critical_tasks,task_count
0,Спринт 2024.3.1.NPP Shared Sprint,96561.384401,43200.0,693310740.0,0.882004,0.024637,1213.664249,5993,1431,12136
1,Спринт 2024.3.2.NPP Shared Sprint,75246.506647,28800.0,611302620.0,0.824293,0.007047,1596.431908,6148,568,12629
2,Спринт 2024.3.3.NPP Shared Sprint,71511.321004,43200.0,541340700.0,0.778597,0.014348,1936.673387,8346,1002,15194
3,Спринт 2024.3.4.NPP Shared Sprint,85791.724138,57600.0,497592000.0,0.818680,0.016747,1392.558655,3350,565,9315
4,Спринт 2024.3.5.NPP Shared Sprint,78228.923848,57600.0,561918360.0,0.806593,0.012555,1310.346761,8908,998,15532


**sprint_name** - Название спринта(по нему происходит агрегация)

**mean_estimation, median_estimation, sum_estimation** - ['mean', 'median', 'sum']  # Средняя, медианная и суммарная оценка времени выполнения задачи (в часах)

**avg_completion_time** - 'mean',  # Среднее время выполнения задач

**completion_rate** - # Доля завершённых задач

**rejected_rate** - # Доля отклонённых задач

**defects** - # Количество дефектов

**critical_tasks** - # Количество критических задач

**task_count** - # Общее количество задач